# Preprocess Data

In [ ]:
from data_processing import dataProcessor

## Alphabets

In [ ]:
# Pre-process data. dataProcessor(source, target, lmap)
dataProcessor("Fine_Tuning/SS_alpha_data", "alphabets/fine_tune_ss", "alpha")

## Numbers

In [ ]:
# Pre-process data. dataProcessor(source, target, lmap)
dataProcessor("Fine_Tuning/SS_num_data", "numbers/num_fine_tune_ss", "num_ft")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

# Load Data For ASL (AlphabetS)

In [ ]:
# Load the data and labels
train_data = np.load("Data/numpy/alphabets/train_data.npy")
train_labels = np.load("Data/numpy/alphabets/train_labels.npy")

test_data = np.load("Data/numpy/alphabets/test_data.npy")
test_labels = np.load("Data/numpy/alphabets/test_labels.npy")

# Reshape data for CNN
train_data = train_data.reshape(-1, 21, 3, 1)
test_data = test_data.reshape(-1, 21, 3, 1)

# Convert labels to categorical format
num_classes = len(np.unique(train_labels))

train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

In [ ]:
# load new data
new_data = np.load('Data/numpy/alphabets/fine_tune_ss_data.npy')
new_labels = np.load('Data/numpy/alphabets/fine_tune_ss_labels.npy')

# reshape data for CNN
new_data = new_data.reshape(-1,21,3,1)

#convert labels to categorical format
new_labels = to_categorical(new_labels, num_classes)

print(new_data.shape, new_labels.shape)

In [ ]:
# Load pre-trained model
model = load_model("models/asl_alphabet_model.keras")

# Load Data For ASL (NumberS)

In [ ]:
# Load the data and labels
train_data = np.load("Data/numpy/numbers/num_train_data.npy")
train_labels = np.load("Data/numpy/numbers/num_train_labels.npy")

test_data = np.load("Data/numpy/numbers/num_test_data.npy")
test_labels = np.load("Data/numpy/numbers/num_test_labels.npy")

# Reshape data for CNN
train_data = train_data.reshape(-1, 21, 3, 1)
test_data = test_data.reshape(-1, 21, 3, 1)

# Convert labels to categorical format
num_classes = len(np.unique(train_labels))

train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

In [ ]:
# load new data
new_data = np.load('Data/numpy/numbers/num_fine_tune_ss_data.npy')
new_labels = np.load('Data/numpy/numbers/num_fine_tune_ss_labels.npy')

# reshape data for CNN
new_data = new_data.reshape(-1,21,3,1)

#convert labels to categorical format
new_labels = to_categorical(new_labels, num_classes)

print(new_data.shape, new_labels.shape)

In [ ]:
# Load pre-trained model
model = load_model("models/asl_number_model.keras")

In [ ]:
# Combine old and new data
combined_data = np.concatenate((train_data, new_data), axis=0)
combined_labels = np.concatenate((train_labels, new_labels), axis=0)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    combined_data, combined_labels, test_size=0.2, shuffle=True, stratify=combined_labels
)

In [ ]:
# Fine-tune the model with a smaller learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [ ]:
# Fine-tune the model
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val,y_val), callbacks=[early_stopping, reduce_lr])

In [ ]:
# Evaluate the model on the test set to check performance
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

In [ ]:
import matplotlib.pyplot as plt

epochs = [i for i in range(100)]
fig,ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(16,9)


ax[0].plot(epochs, train_acc, 'g', label='Training Accuracy')
ax[0].plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()


ax[1].plot(epochs, train_loss, 'g', label='Training Loss')
ax[1].plot(epochs, val_loss, 'r', label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].set_title('Training & Validation Loss')
ax[1].legend()